# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Data Preparation

In [1]:
import torch 
import random
import numpy as np
import os

seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
import pandas as pd

data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [4]:
from zipfile import ZipFile

with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [5]:
from sklearn.model_selection import train_test_split

_, valid = train_test_split(labels, 
                            test_size=0.1,
                            stratify=labels['has_cactus'],
                            random_state=50)

In [6]:
import cv2 
from torch.utils.data import Dataset 

class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__() 
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]    
        img_path = self.img_dir + img_id 
        image = cv2.imread(img_path)     
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        label = self.df.iloc[idx, 1]     

        if self.transform is not None:
            image = self.transform(image) 
        return image, label

In [7]:
from torchvision import transforms 

transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Pad(32, padding_mode='symmetric'),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.Normalize((0.485, 0.456, 0.406),
                                                           (0.229, 0.224, 0.225))])

transform_test= transforms.Compose([transforms.ToTensor(),
                                    transforms.Pad(32, padding_mode='symmetric'),
                                    transforms.Normalize((0.485, 0.456, 0.406),
                                                         (0.229, 0.224, 0.225))])

In [8]:
dataset_train = ImageDataset(df=labels, img_dir='train/', transform=transform_train)
dataset_valid = ImageDataset(df=valid, img_dir='train/', transform=transform_test)

In [9]:
from torch.utils.data import DataLoader 

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

### Create Model

In [10]:
import torch.nn as nn 
import torch.nn.functional as F 

class Model(nn.Module):
    
    def __init__(self):
        super().__init__() 
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(32), 
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))

        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(64),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(128),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(256),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512,
                                              kernel_size=3, padding=2),
                                    nn.BatchNorm2d(512),
                                    nn.LeakyReLU(),
                                    nn.MaxPool2d(kernel_size=2))
        
        self.avg_pool = nn.AvgPool2d(kernel_size=4) 
    
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = x.view(-1, 512 * 1 * 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [11]:
model = Model().to(device)

### Model Training

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006)

In [14]:
epochs = 70 

for epoch in range(epochs):
    epoch_loss = 0 
    
    for images, labels in loader_train:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        epoch_loss += loss.item() 
        loss.backward()
        optimizer.step()
        
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')    

에폭 [1/70] - 손실값: 0.1272
에폭 [2/70] - 손실값: 0.0645
에폭 [3/70] - 손실값: 0.0494
에폭 [4/70] - 손실값: 0.0379
에폭 [5/70] - 손실값: 0.0343
에폭 [6/70] - 손실값: 0.0300
에폭 [7/70] - 손실값: 0.0282
에폭 [8/70] - 손실값: 0.0257
에폭 [9/70] - 손실값: 0.0247
에폭 [10/70] - 손실값: 0.0224
에폭 [11/70] - 손실값: 0.0210
에폭 [12/70] - 손실값: 0.0190
에폭 [13/70] - 손실값: 0.0208
에폭 [14/70] - 손실값: 0.0184
에폭 [15/70] - 손실값: 0.0155
에폭 [16/70] - 손실값: 0.0160
에폭 [17/70] - 손실값: 0.0155
에폭 [18/70] - 손실값: 0.0152
에폭 [19/70] - 손실값: 0.0147
에폭 [20/70] - 손실값: 0.0131
에폭 [21/70] - 손실값: 0.0130
에폭 [22/70] - 손실값: 0.0111
에폭 [23/70] - 손실값: 0.0115
에폭 [24/70] - 손실값: 0.0113
에폭 [25/70] - 손실값: 0.0119
에폭 [26/70] - 손실값: 0.0105
에폭 [27/70] - 손실값: 0.0106
에폭 [28/70] - 손실값: 0.0105
에폭 [29/70] - 손실값: 0.0104
에폭 [30/70] - 손실값: 0.0092
에폭 [31/70] - 손실값: 0.0093
에폭 [32/70] - 손실값: 0.0102
에폭 [33/70] - 손실값: 0.0090
에폭 [34/70] - 손실값: 0.0092
에폭 [35/70] - 손실값: 0.0084
에폭 [36/70] - 손실값: 0.0088
에폭 [37/70] - 손실값: 0.0082
에폭 [38/70] - 손실값: 0.0072
에폭 [39/70] - 손실값: 0.0066
에폭 [40/70] - 손실값: 0.0068
에폭 [41/70

### Model Evaluation

In [15]:
from sklearn.metrics import roc_auc_score

true_list = []
preds_list = []

model.eval() 

with torch.no_grad(): 
    for images, labels in loader_valid:
   
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu() 
     
        preds_list.extend(preds)
        true_list.extend(true)
        
print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}') 

검증 데이터 ROC AUC : 1.0000


### Prediction & submission

In [16]:
dataset_test = ImageDataset(df=submission, img_dir='test/', 
                            transform=transform_test)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

model.eval() 

preds = [] 

with torch.no_grad(): 
    for images, _ in loader_test:
        images = images.to(device)
        
        outputs = model(images)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        preds.extend(preds_part)

In [17]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [18]:
import shutil

shutil.rmtree('./train')
shutil.rmtree('./test')